# Choroid Plexus Segmentation Training via Auto3DSeg

In [55]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import random
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import importlib
from dataclasses import asdict
from loguru import logger
import sys

from reload_recursive import reload_recursive

import mri_data
import monai_training

print_config()

MONAI version: 1.3.2
Numpy version: 1.26.4
Pytorch version: 2.4.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 59a7211070538586369afd4a01eca0a7fe2e742e
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.0
Nibabel version: 5.2.1
scikit-image version: 0.24.0
scipy version: 1.14.0
Pillow version: 10.4.0
Tensorboard version: 2.17.0
gdown version: 5.2.0
TorchVision version: 0.19.0+cu121
tqdm version: 4.66.4
lmdb version: 1.5.1
psutil version: 6.0.0
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.43.3
mlflow version: 2.15.0
pynrrd version: 1.0.0
clearml version: 1.16.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [56]:
reload_recursive(monai_training)
reload_recursive(mri_data)

from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

In [57]:
logger.remove()
logger.add(sys.stderr, level="INFO")

3

In [58]:
load_data = False

In [59]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary3"
work_dir = training_work_dirs / work_dir_name
modalities = ["flair", "t1"]
labels = ["choroid_t1_flair", "pineal", "pituitary"]

## Prep the database

Get the data and labels organized

In [60]:
if load_data:
    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, 'r') as f:
        datalist = json.load(f)

    dataset = preprocess.parse_datalist(datalist_file, dataroot)

In [61]:
if not load_data:
    dataset, _ = preprocess.load_dataset("dataset.json")

In [62]:
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

2024-12-10 23:45:40.029 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/40 [00:00<?, ?it/s]2024-12-10 23:45:40.033 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1010, sesid=20180208)
2024-12-10 23:45:40.055 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
2024-12-10 23:45:40.059 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1011, sesid=20180911)
2024-12-10 23:45:40.075 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1011/ses-20180911/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
2024-12-10 23:45:40.078 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1019, sesid=20190608)
2024-12-10 23:45:40.091 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1019/ses-20190608/choroid_t1_fl

2024-12-10 23:45:40.249 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1109/ses-20180303/choroid_t1_flair-ED.pineal-CH.pituitary-CH.nii.gz
 30%|███       | 12/40 [00:00<00:00, 55.28it/s]2024-12-10 23:45:40.253 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1119, sesid=20161010)
2024-12-10 23:45:40.267 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1119/ses-20161010/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
2024-12-10 23:45:40.270 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1152, sesid=20170529)
2024-12-10 23:45:40.283 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1152/ses-20170529/choroid_t1_flair-ED.pineal-CH.pituitary-SRS.nii.gz
2024-12-10 23:45:40.285 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1163, sesid=20180907)
2024-12-10 23:45:40.299 | SUCCESS  | monai_training.

In the original code, they include labels in the test data as well. Also they have a function that checks that there is nonzero number of voxels in the label

In [63]:
#? I don't know why they put labels for the test data. the brats segmentation code didn't.
# train_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in train_exams]
# test_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in test_exams]
if not load_data:
    train_data = []
    test_data = []
    for scan in dataset:
        if scan.cond == 'tr' and scan.has_label:
            train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
        elif scan.cond == 'ts' and scan.has_label():
            test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


    print(f"Train num total: {len(train_data)}")
    print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


Create and save datalist

In [64]:
if not load_data:
    n_folds = 5
    datalist = {
        "testing": test_data,
        "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
    }

    if not os.path.isdir(work_dir):
        os.makedirs(work_dir)

    # dataroot_dir = "/mnt/h"
    # if not os.path.isdir(dataroot_dir):
    #     os.makedirs(dataroot_dir)

    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, "w") as f:
        json.dump(datalist, f, indent=4)

Load datalist

In [65]:
datalist['testing']

[{'image': '/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1033/ses-20171117/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1033/ses-20171117/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1119/ses-20161010/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1119/ses-20161010/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1259/ses-20200803/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1259/ses-20200803/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1437/ses-20210503/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1437/ses-20210503/choroid_t1_flair-CH.pineal-SRS.pituitary-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1547/ses-20220321/flair

In [66]:
missing_images = []
missing_labels = []
for scan in dataset:
    if not scan.image_path.is_file():
        missing_images.append(scan.image_path)
    if not scan.label_path.is_file():
        missing_images.append(scan.label_path)
print(missing_images)
print(missing_labels)

[]
[]


In [67]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

2024-12-10 23:45:40,971 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3
2024-12-10 23:45:40,975 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/datalist.json.
2024-12-10 23:45:40,975 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/datalist.json.
2024-12-10 23:45:40,977 - INFO - Using user defined command running prefix , will override other settings


In [68]:
max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [69]:
runner.run()

2024-12-10 23:45:40,997 - INFO - Running data analysis...
2024-12-10 23:45:41,000 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [01:17<00:00,  2.58s/it]

2024-12-10 23:46:58,795 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2024-12-10 23:46:58,796 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/datastats.yaml.
2024-12-10 23:46:58,805 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/datastats_by_case.yaml, this may take a while.


2024-12-10 23:46:59,048 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


algo_templates.tar.gz: 104kB [00:00, 260kB/s]                              

...

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2024-12-11 03:23:30,345 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2024-12-11 03:23:30,346 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2024/12/11 03:23:30 INFO mlflow.

2024-12-11 04:15:27,812 - INFO - Ensembling using single GPU!
2024-12-11 04:15:27,813 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/ensemble_output will be used to save ensemble predictions.
2024-12-11 04:15:27,814 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/ensemble_output is created to save ensemble predictions
2024-12-11 04:15:27,854 - INFO - Auto3Dseg picked the following networks to ensemble:
2024-12-11 04:15:27,855 - INFO - swinunetr_0
2024-12-11 04:15:27,856 - INFO - swinunetr_1
2024-12-11 04:15:27,856 - INFO - swinunetr_2
2024-12-11 04:15:27,856 - INFO - swinunetr_3
2024-12-11 04:15:27,857 - INFO - swinunetr_4
2024-12-11 04:15:27,858 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

In [70]:
load_data = False
work_dir_name = "choroid_resegment3"
work_dir = training_work_dirs / work_dir_name
modalities = ["flair", "t1"]
labels = ["choroid_t1_flair"]

In [71]:
if not load_data:
    dataset, _ = preprocess.load_dataset("dataset.json")

In [72]:
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

2024-12-11 04:44:40.512 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/40 [00:00<?, ?it/s]2024-12-11 04:44:40.525 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1010, sesid=20180208)
2024-12-11 04:44:40.533 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1011, sesid=20180911)
2024-12-11 04:44:40.539 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1019, sesid=20190608)
2024-12-11 04:44:40.545 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1029, sesid=20170816)
2024-12-11 04:44:40.550 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1033, sesid=20171117)
2024-12-11 04:44:40.557 | INFO     | monai_training.preprocess:

In [73]:
#? I don't know why they put labels for the test data. the brats segmentation code didn't.
# train_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in train_exams]
# test_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in test_exams]
if not load_data:
    train_data = []
    test_data = []
    for scan in dataset:
        if scan.cond == 'tr' and scan.has_label:
            train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
        elif scan.cond == 'ts' and scan.has_label():
            test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


    print(f"Train num total: {len(train_data)}")
    print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [74]:
if not load_data:
    n_folds = 5
    datalist = {
        "testing": test_data,
        "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
    }

    if not os.path.isdir(work_dir):
        os.makedirs(work_dir)

    # dataroot_dir = "/mnt/h"
    # if not os.path.isdir(dataroot_dir):
    #     os.makedirs(dataroot_dir)

    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, "w") as f:
        json.dump(datalist, f, indent=4)

In [75]:
datalist['testing']

[{'image': '/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208/choroid_t1_flair-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1033/ses-20171117/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1033/ses-20171117/choroid_t1_flair-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1119/ses-20161010/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1119/ses-20161010/choroid_t1_flair-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1259/ses-20200803/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1259/ses-20200803/choroid_t1_flair-ED.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1437/ses-20210503/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1437/ses-20210503/choroid_t1_flair-CH.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1547/ses-20220321/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1547/ses-20220321/choroid_t1_flair-ED.nii.gz'},
 {'image': '/mnt

In [76]:
missing_images = []
missing_labels = []
for scan in dataset:
    if not scan.image_path.is_file():
        missing_images.append(scan.image_path)
    if not scan.label_path.is_file():
        missing_images.append(scan.label_path)
print(missing_images)
print(missing_labels)

[]
[]


In [77]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

2024-12-11 04:44:41,852 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3
2024-12-11 04:44:41,854 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/datalist.json.
2024-12-11 04:44:41,855 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/datalist.json.
2024-12-11 04:44:41,856 - INFO - Using user defined command running prefix , will override other settings


In [78]:
max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [79]:
runner.run()

2024-12-11 04:44:41,876 - INFO - Running data analysis...


2024-12-11 04:44:41,877 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:25<00:00,  1.18it/s]

2024-12-11 04:45:07,294 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2024-12-11 04:45:07,295 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/datastats.yaml.
2024-12-11 04:45:07,302 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/datastats_by_case.yaml, this may take a while.


2024-12-11 04:45:07,476 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2024-12-11 08:27:08,269 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2024-12-11 08:27:08,269 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2024/12/11 08:27:08 INFO mlflow.

2024-12-11 09:21:25,952 - INFO - Ensembling using single GPU!
2024-12-11 09:21:25,956 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/ensemble_output will be used to save ensemble predictions.
2024-12-11 09:21:25,959 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/ensemble_output is created to save ensemble predictions
2024-12-11 09:21:26,034 - INFO - Auto3Dseg picked the following networks to ensemble:
2024-12-11 09:21:26,035 - INFO - swinunetr_0
2024-12-11 09:21:26,036 - INFO - swinunetr_1
2024-12-11 09:21:26,036 - INFO - swinunetr_2
2024-12-11 09:21:26,037 - INFO - swinunetr_3
2024-12-11 09:21:26,038 - INFO - swinunetr_4
2024-12-11 09:21:26,040 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment3/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w